# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [37]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy.crs as ccrs
import geoviews as gv # noqa
import pyproj


# Import API key
from api_keys import geoapify_key

In [38]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv").dropna()


# Display sample data
print(city_data_df.shape)
city_data_df.head()

(571, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,katsuura,35.1333,140.3000,25.51,94,100,3.15,JP,1693776444
1,1,laojunmiao,39.8333,97.7333,21.11,22,100,2.91,CN,1693776444
2,2,novodvinsk,64.4165,40.8122,13.56,85,93,2.18,RU,1693776444
3,3,jinshi,29.6049,111.8701,20.58,87,68,0.91,CN,1693776444
4,4,enewetak,11.3474,162.3373,28.70,74,87,3.37,MH,1693776445


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [39]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", size="Humidity", geo=True, tiles="OSM")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [40]:
#Get user's desired parameters

max_humidity = int(input("Enter the maximum allowed humidity: "))
max_cloudiness = int(input("Enter the maximum allowed cloudiness: "))
max_temperature = int(input("Enter the maximum allowed temperature: "))
min_temperature = int(input("Enter the minimum allowed temperature: "))


Enter the maximum allowed humidity: 25
Enter the maximum allowed cloudiness: 20
Enter the maximum allowed temperature: 35
Enter the minimum allowed temperature: 10


In [43]:
# Narrow down cities that fit criteria and drop any results with null values
acceptable_humidity_s = city_data_df['Humidity'] <= max_humidity
acceptable_cloudiness_s = city_data_df['Cloudiness'] <= max_cloudiness
acceptable_temperature_max_s = city_data_df['Max Temp'] <= max_temperature
acceptable_temperature_min_s = city_data_df['Max Temp'] >= min_temperature

acceptable_weather_s = (acceptable_humidity_s 
             & acceptable_cloudiness_s 
             & acceptable_temperature_max_s 
             & acceptable_temperature_min_s)
acceptable_weather_s

0      False
1      False
2      False
3      False
4      False
       ...  
569    False
570    False
571    False
572    False
573    False
Length: 571, dtype: bool

### Step 3: Create a new DataFrame called `hotel_df`.

In [44]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_columns = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = city_data_df.copy().loc[acceptable_weather_s, hotel_columns]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
44,suwayr,SA,30.1171,40.3893,15,None
55,at taj,LY,24.2049,23.2857,22,None
125,bechar,DZ,31.6167,-2.2167,11,None
140,calingasta,AR,-31.3308,-69.4078,11,None
194,ciudad melchor muzquiz,MX,27.8833,-101.5167,22,None
218,qaisumah,SA,28.3112,46.1273,19,None
219,murzuq,LY,25.9155,13.9184,15,None
263,tarinkot,AF,32.6268,65.8733,12,None
265,paghman,AF,34.5875,68.9533,23,None
274,as sulayyil,SA,20.4607,45.5779,13,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [47]:
# Set parameters to search for a hotel
radius = 50000
params = {
    "categories": "hotel",
    "apiKey": geoapify_key,
    "limit": 200
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
suwayr - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
bechar - nearest hotel: No hotel found
calingasta - nearest hotel: No hotel found
ciudad melchor muzquiz - nearest hotel: No hotel found
qaisumah - nearest hotel: No hotel found
murzuq - nearest hotel: No hotel found
tarinkot - nearest hotel: No hotel found
paghman - nearest hotel: No hotel found
as sulayyil - nearest hotel: No hotel found
kerman - nearest hotel: No hotel found
san juan - nearest hotel: No hotel found
sabha - nearest hotel: No hotel found
yangiyul - nearest hotel: No hotel found
huambo - nearest hotel: No hotel found
sarina - nearest hotel: No hotel found
adrar - nearest hotel: No hotel found
shibirghan - nearest hotel: No hotel found
encarnacion de diaz - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
44,suwayr,SA,30.1171,40.3893,15,No hotel found
55,at taj,LY,24.2049,23.2857,22,No hotel found
125,bechar,DZ,31.6167,-2.2167,11,No hotel found
140,calingasta,AR,-31.3308,-69.4078,11,No hotel found
194,ciudad melchor muzquiz,MX,27.8833,-101.5167,22,No hotel found
218,qaisumah,SA,28.3112,46.1273,19,No hotel found
219,murzuq,LY,25.9155,13.9184,15,No hotel found
263,tarinkot,AF,32.6268,65.8733,12,No hotel found
265,paghman,AF,34.5875,68.9533,23,No hotel found
274,as sulayyil,SA,20.4607,45.5779,13,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [49]:
%%capture --no-display

# Configure the map plot
map_hotel_plot = hotel_df.hvplot.points("Lng", "Lat", size="Humidity", geo=True, tiles="OSM")

# Display the map
map_hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)